In [1]:
from astropy.io import fits, ascii
from astropy.wcs import WCS
import numpy as np
import matplotlib.pyplot as plt
import os, time
from photutils import DAOStarFinder
from astropy.table import QTable
from astropy.stats import mad_std
from photutils import aperture_photometry, CircularAperture, CircularAnnulus
from a345_utilities import print_header         # to use this in your own scripts copy the file a345_utilities.py to your work directory
import astropy.units as unit
from astropy.wcs.utils import pixel_to_skycoord
from photutils import CircularAperture, CircularAnnulus, SkyCircularAperture, aperture_photometry
from matplotlib.colors import LogNorm
from photutils import IRAFStarFinder
from astropy.stats import sigma_clipped_stats
from os import listdir

In [2]:
class Star:
    def __init__(self, ra, dec):
        self.ra = ra
        self.dec = dec
        self.r = Filter()
        self.g = Filter()
        self.i = Filter()

    

class Filter:
        
    def __init__(self): 
        self.flux = []
        self.mag = []
        self.avg_mag = 0
        self.avg_flux = 0
            
    def add(self, flux, mag):
        self.flux.append(flux)
        self.mag.append(mag)
        
        if not np.isnan(mag):
            self.avg_mag = np.mean(self.mag)
            self.avg_flux = np.mean(self.flux)

In [4]:
src = "M35"
size = "3000x3000"

# Specify the platesolved image output path
Input_directory = '/data/observatory/student_data/Alasdairs File/Output_Files/Cluster Photometry/M35/'

# Specify the platesolved image output path
#Output_directory = 'examples/a345/lab_projects/shared-data/student_data/Alasdairs File/Output_Files'

# folderpath = os.path.join(Input_directory)
folderpath = Input_directory

filenames = [f for f in listdir(folderpath) if f.endswith('.fits')]

filenames.sort()

stars = {}

table = np.genfromtxt(f'/data/observatory/student_data/natalia_bajnokova/M35/Calibrated_3000x3000/data_cluster_stars_M35.dat', names=True,dtype=None)
ra = table['RA']
dec = table['DEC']

for file in (filenames):
    #import fit
    print (file)
    with fits.open(folderpath+file) as hdu:
        header = hdu[0].header
        data = hdu[0].data
    
    filtr = header["FILTER"][0]
    wcs = WCS(header)
    bkg_sigma = mad_std(data)        # get a measure of the image noise level
    SeeingStars = IRAFStarFinder(fwhm=8, threshold=50*bkg_sigma, brightest=250)     # set the detection threshold for a source based on the image noise
    SeeingStars = SeeingStars(data)
    fwhm = np.mean(SeeingStars['fwhm']) # calculates the average size of the stars in the image

    # convert ra and dec positions to pixel positions in the image
    xc,yc = wcs.wcs_world2pix(ra, dec, 1)
    PixPos = np.transpose((xc,yc))
    
    # sets the aperature and radii to use
    pix_radius = 4*fwhm/2 # 4 times the average star raduis to collect all the possible photons
    In_ann = pix_radius*1.2; Out_ann = (In_ann+1)*1.3 # annulus measurements from 1.2*aperture to 1.3*aperture
    
    # aperture photometry data
    aperture = CircularAperture(PixPos, r=pix_radius) # creating the cicular apertures cnetred on the star positions
    annulus_aperture = CircularAnnulus(PixPos, r_in=In_ann, r_out=Out_ann) # creating the annulus apertures cnetred on the star positions
    apers = [aperture, annulus_aperture]
    # Send the created apertures and the calibrated data to do the photometry
    print('stars and apertures created with wcs_world2pix, starting aperture photometry')
    phot_table = aperture_photometry(data, apers)
    
    phot_table['RA'] = ra; phot_table['DEC'] = dec                  # add to the table columns for Ra and Dec coordinates of the stars
    bkg_mean = phot_table['aperture_sum_1'] / annulus_aperture.area # average value of the annulus around each star
    phot_table['background_mean'] = bkg_mean                        # adding the background level to the output table
    bkg_sum = bkg_mean * aperture.area                              # background for each star
    final_sum = phot_table['aperture_sum_0'] - bkg_sum              # The actual photometry results
    filter_counts_name = 'counts_'+header['FILTER']
    filter_flux_name = 'flux_'+header['FILTER']
    phot_table[filter_counts_name] = final_sum                      #  adding results to the output table
    filter_flux = final_sum/header['EXPTIME']
    phot_table[filter_flux_name] = filter_flux
    phot_table[f"mag {header['FILTER']}"] =-2.5*np.log10(filter_flux)

    """
    #Plotting

    fig = plt.figure(figsize=(20,20))
    plt.imshow(data, cmap="gray", norm=LogNorm(vmin = bkg_mean, vmax=bkg_mean+200) )
    star_aperature.plot(color="red")
    annulus_aperature.plot(color="green")
    plt.show()
    """


    
    ra, dec = wcs.wcs_pix2world(phot_table["xcenter"],phot_table["ycenter"], 1)

    flux = phot_table[filter_flux_name]
    count = phot_table[filter_counts_name]
    mag = phot_table[f"mag {header['FILTER']}"]
    tbl = QTable([ra , dec, flux, count, mag], names=('RA','DEC', "flux", "count", "mag"), masked=True)
    tbl.sort("flux")
    tbl.reverse()
    
    ascii.write(tbl, f'/data/observatory/student_data/natalia_bajnokova/{src}/Calibrated_{size}/{filtr}/data_{file}.dat')
    for row in tbl:
        
        if row["mag"] == "nan":
            pass
        else:
            r = round(row["RA"], 4)
            d = round(row["DEC"], 4)
            k = f"{r}, {d}"
            if k in stars:
                if filtr == "G":
                    obj = stars[k]
                    obj.g.add(row["flux"], row["mag"])
                elif filtr == "I":
                    obj = stars[k]
                    obj.i.add(row["flux"], row["mag"])
                else:
                    obj = stars[k]
                    obj.r.add(row["flux"], row["mag"])
            else:
                if filtr == "G":
                    stars[k] = Star(r, d)
                    stars[k].g.add(row["flux"], row["mag"])
                elif filtr == "I":
                    stars[k] = Star(r, d)
                    stars[k].i.add(row["flux"], row["mag"])
                else:
                    stars[k] = Star(r, d)
                    stars[k].r.add(row["flux"], row["mag"])

SolCal_M35_LIGHT_G_120_-5C_20210202_233233_768_E_0001.fits
stars and apertures created with wcs_world2pix, starting aperture photometry


<ipython-input-4-0ac2f41d0552>:63: RuntimeWarning: invalid value encountered in log10
  phot_table[f"mag {header['FILTER']}"] =-2.5*np.log10(filter_flux)


SolCal_M35_LIGHT_G_120_-5C_20210202_233505_721_E_0002.fits
stars and apertures created with wcs_world2pix, starting aperture photometry


<ipython-input-4-0ac2f41d0552>:63: RuntimeWarning: invalid value encountered in log10
  phot_table[f"mag {header['FILTER']}"] =-2.5*np.log10(filter_flux)


SolCal_M35_LIGHT_G_120_-5C_20210202_233712_979_E_0003.fits
stars and apertures created with wcs_world2pix, starting aperture photometry


...

SolCal_M35_LIGHT_R_120_-5C_20210203_010432_665_E_0019.fits
stars and apertures created with wcs_world2pix, starting aperture photometry


<ipython-input-4-0ac2f41d0552>:63: RuntimeWarning: invalid value encountered in log10
  phot_table[f"mag {header['FILTER']}"] =-2.5*np.log10(filter_flux)


SolCal_M35_LIGHT_R_120_-5C_20210203_010701_032_E_0020.fits
stars and apertures created with wcs_world2pix, starting aperture photometry


<ipython-input-4-0ac2f41d0552>:63: RuntimeWarning: invalid value encountered in log10
  phot_table[f"mag {header['FILTER']}"] =-2.5*np.log10(filter_flux)


SyntaxError: 'return' outside function (<ipython-input-4-0ac2f41d0552>, line 115)